In [36]:
import torch

device = torch.device('cuda')

In [37]:
# dataset = load_dataset("text", data_files={"train": 'data/merged_delete_later.txt'})

In [38]:
# eval_dataset = load_dataset('opus100', 'en-ru', split='train')

In [39]:
model_path = 'C:/tmp/tst-translation/kbd_lat-ru_t5-base_char_tok/checkpoint-4000'
# model_path = 'google/mt5-small'

In [40]:
from transformers import T5Tokenizer

# tokenizer = T5TokenizerFast.from_pretrained('kbd-ru-t5-small', extra_ids=0)
tokenizer = T5Tokenizer.from_pretrained(model_path, extra_ids=0)

In [41]:
def translate(input_text):
    # from bleu_eval import sacre_bleu_score

    # # input_text, target_text = dataset['train'][random.randint(0, len(dataset['train']) - 1)]['text'].split('😀')
    #
    # # target_text = 'test'
    # # eval_lines = re.split('(?<=[.!?]) +', eval_dataset['train'][random.randint(0, len(eval_dataset['train']) - 1)]['text'])
    # # input_text = eval_lines[random.randint(0, len(eval_lines)-1)]
    # eval_lines = eval_dataset[random.randint(0, len(eval_dataset) - 1)]['translation']
    # input_text =eval_lines['ru']
    # # target_text =eval_lines['en']

    # input_text = "мужчина ушел в город."
    # target_text = "When I looked again I noticed something shiny."
    encodings = tokenizer(
        'kbd->ru: ' + input_text,
        # 'ru->kbd: ' + input_text,
        # + "</s>",
        padding='longest',
        max_length=256,
        truncation=True,
        return_tensors="pt")

    encodings.to(device)
    input_ids = encodings.input_ids

    # input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(['▁kb', 'd', '-', '>', 'ru', '>', '▁', '▁', 'uə', '</s>'])], device=device)
    #
    # print(input_ids)

    input_ids.to(device)
    model.to(device)


    n_beams = 2
    # for i in range(1, n_beams + 1):
    output_tokens = model.generate(input_ids,
                                   num_beams=n_beams,
                                   num_return_sequences=n_beams,
                                   # output_attentions=True,
                                   # output_scores = True,
                                   # repetition_penalty = 5.5,
                                   # length_penalty=0.2,
                                   # do_sample=True,
                                   # temperature=0.1,
                                   # bos_token_id = 2,
                                   # output_scores=True,
                                   # max_length=128
                                   )

    # output_tokens[output_tokens==0]=1 # replace <unk> with <s> (— bos token)

    # print(str(i) + ' ' + input_text + '  ->  ' + tokenizer.decode(output_tokens[0], skip_special_tokens=True))
    # print('n_beams: ' + str(i))
    print('n_beams: ' + str(n_beams))
    print('input: ' + input_text + '\n')
    print(tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text)))
    for token_set in output_tokens:
        prediction = tokenizer.decode(token_set, skip_special_tokens=True)
        # print(token_set)
        print("output: {}".format(prediction))
        # print('BLEU score: ' + str(sacre_bleu_score(predictions=prediction, references=target_text)['score']) + '\n')
    # print('reference: ' + target_text)

In [42]:
from transformers import T5ForConditionalGeneration

# model = MT5ForConditionalGeneration.from_pretrained('models/' + model_path + '/epoch_9')
model = T5ForConditionalGeneration.from_pretrained('C:/tmp/tst-translation')
model.to(device)

# model.config.vocab_size = len(tokenizer.get_vocab())  # remove later, bug fixing

T5ForConditionalGeneration(
  (shared): Embedding(8000, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(8000, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): Li

In [43]:
def cyrillic_to_latin(text):
    with open('../data/kbd cyrillic-latin alphabet table.txt', 'r', encoding='utf-8') as alphabet_table:
        for line in alphabet_table:
            key, value = line.split(':')
            text = text.replace(key, value.replace('\n', ''))
    return text

In [44]:
inp = 'псы дахэхэр'
# inp = 'сэ унэ дахэ сиIэщ.'
# inp = 'уи унэр дахэщ'
# inp = 'мы псыр псынщIэщ.'
# inp = 'щIалэм сабийр къиIэтащ'
# inp = 'щIалэр губзыгъэщ'
# inp = 'игъэщIэгъуащ'
# inp = 'укъэкIуэнущ'
# inp = 'псалъэ дахэхэр'
# inp = 'псалъэ ныкъуэ'
# inp = 'псышхуэ'
# inp = 'унэр псыфщ'
# inp = 'къэкIуэнур къакIуэри къэкIуахэщ къэвгъакIуэ къэвгъэкIуащ къэзгъэкIуащ къэдгъэкIуащ'
# inp = 'джанэр щIитIэгъащ.'
inp = 'уэшх IэфI'
# inp = 'шы плъыжь'
# inp = 'плъыжьышхуэ'
# inp = 'мы унэр плъагъурэ?'
# inp = 'унэр слъэгъуащ'
# inp = 'тхылъышхуэ'
# inp = 'тхылъ цIыкIум уеджащ'
# inp = 'тхылъ цIыкIунитIэ'
# inp = 'жыг дахэ'
# inp = 'махуэ, мазэ, гъэ'
# inp = 'махуэ, мазэ, илъэс'
# inp = 'сыт апхуэдэу щIыжыпIэр?'
# inp = 'апхуэдэу щхьэ жыпIэрэ?'
# inp = 'сыт?'
# inp = 'хуит ищIащ'
# inp = 'КъыкIэлъыкIуэнущ'
# inp = 'ар къакIуэри псэлъащ'
# inp = 'пщэдей къызэптыжмэ, тхылъыр уэстынущ'
# inp = 'ар къакIуэри жиIащ: "псы сефэнут".'
# inp = 'ар къакIуэри жиIащ: "псы сефэнущ".'
# inp = 'ар нэхъыфIщ'
# inp = 'ар нэхъ дахэщ'
# inp = 'махуитI'
# inp = 'бгы лъагэ'
# inp = 'шу закъуэ'
# inp = 'пщащэ цIыкIу'
# inp = 'мэл хужьхэр'
# inp = 'дэнэ сыздэкIуэр?'
# inp = 'зы мэл гуэр'
# inp = 'фыхуеймэ фыкъэкIуэж'
# inp = 'абы щыгъуэ ахэр хадэм щылэжьэрт'
# inp = 'мэзым дыдыхьащ'
# inp = 'си пщащэ цIыкIу'
# inp = 'уи Iуэху здэщымыIэ умыкIуэ'
# inp = 'уи Iуэху здэщымыIэм умыкIуэ'
# inp = 'ЦIыхухэр я закъуэу псэукъым'
# inp = 'хабзэ дахэ'
# inp = 'цIыхум уз къыхихыным хуэдэу'
# inp = 'уэ дэнэ ущыIэ?'
# inp = 'дышхэнукъэ?'
# inp = 'деплъынукъэ?'
# inp = 'унэхэр инщ'
# inp = 'дэ къалэм дыщопсэу'
# inp = 'абы щыгъуэ мэзым сыщыпсэурт'
# inp = 'сышхэри сыжеижащ'
# inp = 'сыздэщыIэр дэнэ?'
# inp = 'уздэщыIэр дэнэ?'
# inp = 'кIуэи жей'
# inp = 'мэлхэр мэзым щоджэгу'
# inp = 'хьэхэм мыIэрысэхэр яшхыркъым'
# inp = 'адыгэ'
inp = 'тIури йогупсыс'
# inp = 'ирагъэлъагъуну'

translate(cyrillic_to_latin(inp))
# translate(inp)

# translate('умный дом')
# translate('твой лес')
# translate('забежал в дом')
# translate('ты кто?')
# translate('я не знаю')
# translate('Они любили друг друга.')
# translate('кто-то забежал в дом.')
# translate('лошадь бежит с горы')
# translate('покушать вместе с нами')
# translate('я и они')
# translate('Первые сорок дней с ним был мальчик')

#upload the ru-kbd model

n_beams: 2
input: ṯuri yogupsıs

['▁', 'ṯ', 'u', 'r', 'i', '▁', 'y', 'o', 'g', 'u', 'p', 's', 'ı', 's', '</s>']
output: к<extra_id_-557>j<extra_id_-1148><extra_id_-53>ца<extra_id_-207>u<extra_id_-175><extra_id_-186><extra_id_-1369>ä<extra_id_-6529><extra_id_-1881>t к<extra_id_-852>
output: к<extra_id_-557>j<extra_id_-1148><extra_id_-53>ца<extra_id_-207>u<extra_id_-175><extra_id_-186><extra_id_-1369>ä<extra_id_-6529><extra_id_-1881>t кα
